
##### This is a Kaggle Jupyter Notebook. 
##### First, this script connects to the API Football data to fetch the Premier League standings results in a json file (Extract). 
##### Next, the useful data is arranged into a list of tuples which can be parsed into a pandas dataframe (Transform). 
##### Finally, the script connects to a database in Azure Database for MySQL to update a table with the standings results.


In [26]:
# load all necessary libraries

!pip install mysql-connector-python # install mysql-connector-python package to connect with MySQL 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import requests
import mysql.connector
from kaggle_secrets import UserSecretsClient # enable kaggle secrets for this notebook to replace .env file


In [27]:
# retrieve credentials to connect API

API_KEY = UserSecretsClient().get_secret("API_KEY")
API_HOST = UserSecretsClient().get_secret("API_HOST")

url = API_HOST
headers = {"X-Auth-Token": API_KEY}

**EXTRACT DATA FROM API ---------------------------------------------------------------------------------------------------------**

In [28]:
# fetch data from API

response = requests.get(url, headers = headers).json()


In [29]:
# filter the response down to the standings table list

standings_list = response["standings"][0]["table"]


**TRANSFORM----------------------------------------------------------------------------------------------------------------------**

In [30]:
# arrange clubs records into a list of tuples

rows = [] 

for club in standings_list:
    season = 2025
    position = club["position"]
    team_id = club["team"]["id"]
    team = club["team"]["name"]
    played = club["playedGames"]
    won = club["won"]
    draw = club["draw"]
    lost = club["lost"]
    goals_for = club["goalsFor"]
    goals_against = club["goalsAgainst"]
    goal_diff = club["goalDifference"]
    points = club["points"]
    form = club["form"]
    
    club_records = (season, position, team_id, team, played, won, draw, lost, goals_for, goals_against, goal_diff, points, form)
    rows.append(club_records)


In [31]:
# create pandas DataFrame

column_names = ["season", "position", "team_id", "team", "played", "won", "draw", "lost", "goals_for", "goals_against", "goal_diff", "points", "form"]

standings_df = pd.DataFrame(rows, columns = column_names)
standings_df

,season,position,team_id,team,played,won,draw,lost,goals_for,goals_against,goal_diff,points,form
0,2025,1,64,Liverpool FC,3,3,0,0,8,4,4,9,None
1,2025,2,61,Chelsea FC,3,2,1,0,7,1,6,7,None
2,2025,3,57,Arsenal FC,3,2,0,1,6,1,5,6,None
3,2025,4,73,Tottenham Hotspur FC,3,2,0,1,5,1,4,6,None
4,2025,5,62,Everton FC,3,2,0,1,5,3,2,6,None
5,2025,5,71,Sunderland AFC,3,2,0,1,5,3,2,6,None
6,2025,7,1044,AFC Bournemouth,3,2,0,1,4,4,0,6,None
7,2025,8,354,Crystal Palace FC,3,1,2,0,4,1,3,5,None
8,2025,9,66,Manchester United FC,3,1,1,1,4,4,0,4,None
9,2025,10,351,Nottingham Forest FC,3,1,1,1,4,5,-1,4,None


**LOAD---------------------------------------------------------------------------------------------------------------------------**

In [32]:
# retrieve credentials to connect to MySQL

MYSQL_HOST = UserSecretsClient().get_secret("MYSQL_HOST")
MYSQL_PORT = UserSecretsClient().get_secret("MYSQL_PORT")
MYSQL_USER = UserSecretsClient().get_secret("MYSQL_USER")
MYSQL_PWD = UserSecretsClient().get_secret("MYSQL_PWD")
MYSQL_DB = UserSecretsClient().get_secret("MYSQL_DB")

In [33]:
# test connection to MySQL server

server_conn = mysql.connector.connect(
    host = MYSQL_HOST,
    port = MYSQL_PORT,
    user = MYSQL_USER,
    password = MYSQL_PWD,
    ssl_ca = "/kaggle/input/ca-cert/DigiCertGlobalRootG2.crt.pem", ssl_disabled = False,
    connection_timeout = 10, # process will stops after 10asec without a response
    raise_on_warnings = True # turns MySQL errors into python exceptions
)
server_cur = server_conn.cursor()
print(f"[SUCCESS] - Connected to MySQL server!")

# close connection to server
server_cur.close()
server_conn.close()

[SUCCESS] - Connected to MySQL server!


In [34]:
# connect to premier_league_db database

db_conn = mysql.connector.connect(
    host = MYSQL_HOST,
    port = MYSQL_PORT,
    user = MYSQL_USER,
    password = MYSQL_PWD,
    database = MYSQL_DB,
    ssl_ca = "/kaggle/input/ca-cert/DigiCertGlobalRootG2.crt.pem", ssl_disabled = False,
    connection_timeout = 10, # process will stops after 10asec without a response
    autocommit = False, # transactions are written in the database ONLY when we call a commit operation
    raise_on_warnings = True # turns MySQL errors into python exceptions
)
cursor = db_conn.cursor()
print(f"Connection to database successful!")

Connection to database successful!


In [35]:
# test if table exists in database

sql_table = "standings"
cursor.execute("SHOW TABLES LIKE %s", (f"{sql_table}",))

if cursor.fetchone() is None:
    raise SystemExit(f"This table '{sql_table}' does not exist...please create it")
else:
    print(f"[SUCCESS] - The table '{sql_table}' exists... proceed to next step")

[SUCCESS] - The table 'standings' exists... proceed to next step


In [36]:
# number of rows in table

no_rows = len(rows)

# update columns with variable values

UPSERT_SQL = (f"""
INSERT INTO {sql_table}
(season, position, team_id, team, played, won, draw, lost, goals_for, goals_against, goal_diff, points, form)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) AS src
ON DUPLICATE KEY UPDATE
position = src.position,
team = src.team,
played = src.played,
won = src.won,
draw = src.draw,
lost = src.lost,
goals_for = src.goals_for,
goals_against = src.goals_against,
goal_diff = src.goal_diff,
points = src.points,
form = src.form;"""
)


In [37]:
# run UPSERT operation in MySQL

try:
    cursor.executemany(UPSERT_SQL, rows)
    db_conn.commit()
    print(f"[SUCCESS] - Upsert completed for {no_rows} rows")
except Exception as e:
    db_conn.rollback()
    print(f"[ERROR] - Rolled back due to the following error: {e}")
finally:
    cursor.close()
    db_conn.close()
    print("\nAll database connections closed. \nClean up completed.")

[SUCCESS] - Upsert completed for 20 rows

All database connections closed. 
Clean up completed.
